In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/

In [63]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

from src.utils.databricks_pp import handle_databricks_cost
from src.utils.azure_pp import handle_azure_costs
from src.utils.aws_pp import handle_aws_costs

In [10]:
from_date = "2024-01-01"
to_date="2025-06-12"
timestamp = "2025-06-13 11:40:40.049356"
path_to_dir = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/{from_date}_to_{to_date}/{timestamp}/"
path_to_data=f"{path_to_dir}final_costs_with_azure_and_aws_and_db_{from_date}_{to_date}.csv"
path_to_save = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/infrastructure_insights/"

In [11]:
os.makedirs(path_to_save, exist_ok=True)

# Load Data

In [ ]:
df = pd.read_csv(path_to_data)
df = df[[col for col in df.columns if "Unnamed" not in col]]


# PP Data

In [20]:
df = df[df["cloud"].isnull() == False]

In [21]:
df = handle_databricks_cost(df)
df = handle_aws_costs(df)
df = handle_azure_costs(df)
df

# Get unique serviceName per account - grouped by 

In [43]:
min_date = df["date"].min()
max_date = pd.to_datetime(df["date"], format="mixed", utc=True).dt.strftime("%Y-%m-%d").max()
print(min_date)
print(max_date)

In [30]:
cloud_services = df.groupby(by=["cloud","serviceCategoryName","serviceName"])["date"].count().reset_index().sort_values(by=["cloud","serviceCategoryName","serviceName"],ascending=True)
cloud_services

In [45]:
cloud_services.to_csv(f"{path_to_save}unique_cloud_services_{min_date}_{max_date}_{datetime.now()}.csv")

In [48]:
cloud_services.groupby(by="cloud")["serviceCategoryName"].nunique()

# Load labeled list via ChatGPT

In [61]:
df_labeled = pd.read_csv(f"{path_to_save}Cost_Data_with_Detailed_Documentation_Links_v2.csv",sep=";", index_col=0)

In [62]:
df_labeled

In [64]:
df_labeled["serviceName_without_country_string"] = df_labeled["serviceName"].str.split("-").str[1::].str.join("-")
df_labeled["serviceName_without_country_string"] = np.where(
    df_labeled["serviceName_without_country_string"].str.contains("AWS-Out-Bytes"),
    "AWS-Out-Bytes",
    df_labeled["serviceName_without_country_string"],
)
df_labeled["serviceName_without_country_string"] = np.where(
    df_labeled["serviceName_without_country_string"].str.contains("AWS-In-Bytes"),
    "AWS-In-Bytes",
    df_labeled["serviceName_without_country_string"],
)
df_labeled["serviceName_without_country_string"].value_counts()

In [65]:
df_labeled.sort_values(by=["cloud","serviceName_without_country_string"]).to_csv(f"{path_to_save}Cost_Data_with_Detailed_Documentation_Links_without_contry_strings.csv",sep=";")